# LEVEL 1 — Full-System Hemodynamics Validation
Goal: To verify that the entire cardiovascular system behaves physiologically:
- aortic pressure
- aortic flow
- cardiac output
- heart rate
- waveform periodicity
- comparison Abel_ref2 vs cow_runV23

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

RESULTS_DIR = "../projects/simple_run/results"

RUNS = {
    "Abel_ref2": "Abel_ref2",
    "cow_runV23": "cow_runV23",
}

P_FILE = "aorta.txt"
Q_FILE = "R_lv_aorta.txt"
G_FILE = "g.txt"

In [ ]:
def load_ts(path):
    raw = open(path).read().splitlines()
    data = []
    for line in raw:
        line = line.replace(",", " ").replace(";", " ").strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) < 2:
            continue
        try:
            t = float(parts[0])
            v = float(parts[1])
            data.append([t, v])
        except:
            continue
    arr = np.array(data)
    return arr[:,0], arr[:,1]

def compute_level1_metrics(run_name):
    heart_dir = os.path.join(RESULTS_DIR, run_name, "heart_kim_lit")

    # load raw time series
    t_p, p_raw = load_ts(os.path.join(heart_dir, P_FILE))
    t_q, q_raw = load_ts(os.path.join(heart_dir, Q_FILE))

    # ground pressure (atmospheric)
    g_path = os.path.join(heart_dir, G_FILE)
    if os.path.exists(g_path):
        _, g = load_ts(g_path)
        p = p_raw - g
    else:
        p = p_raw - 1e5    # fallback as in FirstBlood

    # unit conversions
    p_mmHg = p / 133.322
    q_Lmin = q_raw * 60.0 * 1000.0

    # detect heartbeats
    peaks, _ = find_peaks(p_mmHg, distance=40)
    HR = np.nan
    if len(peaks) > 1:
        dt = np.diff(t_p[peaks]).mean()
        HR = 60.0 / dt

    # cardiac output = mean flow
    CO = np.mean(q_Lmin)

    # basic stats
    systolic  = np.max(p_mmHg)
    diastolic = np.min(p_mmHg)
    mean_p    = np.mean(p_mmHg)
    pulse_p   = systolic - diastolic

    # periodicity check: last 2 cycles
    if len(peaks) >= 3:
        i1, i2 = peaks[-3], peaks[-2]
        j1, j2 = peaks[-2], peaks[-1]
        seg1 = p_mmHg[i1:i2]
        seg2 = p_mmHg[j1:j2]
        m = min(len(seg1), len(seg2))
        periodicity_rms = np.sqrt(np.mean((seg1[:m] - seg2[:m])**2))
    else:
        periodicity_rms = np.nan

    return {
        "run": run_name,
        "SBP": systolic,
        "DBP": diastolic,
        "MAP": mean_p,
        "PulsePressure": pulse_p,
        "HR": HR,
        "CO": CO,
        "PeriodicityRMS": periodicity_rms,
        "t_p": t_p,
        "p_mmHg": p_mmHg,
        "t_q": t_q,
        "q_Lmin": q_Lmin,
    }


In [ ]:
records = []

results = {}
for label, folder in RUNS.items():
    res = compute_level1_metrics(folder)
    results[label] = res
    records.append({
        "Run": label,
        "SBP (mmHg)": res["SBP"],
        "DBP (mmHg)": res["DBP"],
        "MAP (mmHg)": res["MAP"],
        "PulseP": res["PulsePressure"],
        "HR (bpm)": res["HR"],
        "CO (L/min)": res["CO"],
        "Periodicity RMS (mmHg)": res["PeriodicityRMS"],
    })

df_L1 = pd.DataFrame(records)
df_L1


In [ ]:
plt.figure(figsize=(12,5))

for label, res in results.items():
    plt.plot(res["t_p"], res["p_mmHg"], label=label)

plt.title("Aortic Pressure Waveform (mmHg)")
plt.xlabel("Time (s)")
plt.ylabel("Pressure (mmHg)")
plt.grid(True)
plt.legend()
plt.show()
